# Codon Bias

In [1]:
library(coRdon)

### Load data

In [2]:
genomes <- read.table("codBias/list.txt", header = FALSE, quote="", col.names = 'genomes')
head(genomes, 5)

,genomes
,<chr>
1,G000005825
2,G000006175
3,G000006605
4,G000006725
5,G000006745


In [3]:
dim(genomes)

[1] 10575     1

### Demo

In [4]:
genomes[1, 'genomes']

[1] "G000005825"

Load cds and hegs (concatenated)

In [5]:
seqs <- readSet(file = paste('codBias/concatenated/', genomes[1, 'genomes'], '.fa', sep = ''))

Convert to codon table

In [6]:
ct <- codonTable(seqs)

Calculate [ENCprime](https://academic.oup.com/mbe/article/19/8/1390/997706?login=true) (codon bias measure) for all the cds and hegs.

In [7]:
cub <- ENCprime(ct, filtering = 'hard', id_or_name2 = '11')

Calculate $\Delta$ENCprime as described by [Vieira-SIlva (2010)](https://journals.plos.org/plosgenetics/article?id=10.1371/journal.pgen.1000808).

In [8]:
startTime <- Sys.time()
delta <- 1 - (cub[2]/ cub[1])
delta
endTime <- Sys.time()
print(endTime - startTime)

[1] 0.2941715

Time difference of 0.008624792 secs


In [9]:
delta_cub <- function(g){
    # Load concatenated cds and hegs
    seqs <- readSet(file = paste('codBias/concatenated/', g, '.fa', sep = ''))
    
    # Convert to codon tables
    ct <- codonTable(seqs)
    
    # MILC
    cub_milc <- MILC(ct, filtering = 'hard', id_or_name2 = '11')
    # ENCprime
    cub_encp <- ENCprime(ct, filtering = 'hard', id_or_name2 = '11')
    
    # Calculate delta
    delta_milc <- 1 - (cub_milc[2]/ cub_milc[1])
    delta_encp <- 1 - (cub_encp[2]/ cub_encp[1])
    
    return(c(delta_milc, delta_encp))
}

In [11]:
startTime <- Sys.time()
deltas <- sapply(genomes$genomes, delta_cub)
endTime <- Sys.time()
print(endTime - startTime)

Time difference of 58.41703 mins


Reformat output

In [12]:
deltas <- matrix(deltas, nrow = length(genomes$genomes), byrow = TRUE)
dim(deltas)

[1] 10575     2

Write table

In [13]:
write.table(data.frame(x = genomes$genomes, y = deltas), "cbias_corrected.tsv", sep= "\t", col.names = FALSE, 
            row.names = FALSE, quote = FALSE)